In [19]:
import os
from sqlalchemy import create_engine
import pybaseball as pyb
import pybaseball.cache # Ensure caching is imported
import pandas as pd
from dotenv import load_dotenv
import time
from datetime import date, timedelta
from sqlalchemy.engine import Engine
from sqlalchemy import text
import datetime
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException
from rapidfuzz import process
import re
import numpy as np
from io import StringIO
import pylahman
import statsapi

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

Get probable starter of the day

In [7]:
schedule = statsapi.schedule(date=  '6/15/2025')

In [8]:
params = {
        'date': '6/15/2025',
        'sportId': 1,
        'hydrate': 'team' 
    }
data = statsapi.get('schedule', params)

Details of the different gamestatus

In [6]:
a = statsapi.meta(type="gameStatus")

In [24]:
import statsapi
import pandas as pd
import pytz
from datetime import datetime

# Team abbreviations to keep the table mobile-friendly
TEAM_ABBR = {
    'Arizona Diamondbacks': 'ARI', 'Atlanta Braves': 'ATL', 'Baltimore Orioles': 'BAL',
    'Boston Red Sox': 'BOS', 'Chicago White Sox': 'CWS', 'Chicago Cubs': 'CHC',
    'Cincinnati Reds': 'CIN', 'Cleveland Guardians': 'CLE', 'Colorado Rockies': 'COL',
    'Detroit Tigers': 'DET', 'Houston Astros': 'HOU', 'Kansas City Royals': 'KC',
    'Los Angeles Angels': 'LAA', 'Los Angeles Dodgers': 'LAD', 'Miami Marlins': 'MIA',
    'Milwaukee Brewers': 'MIL', 'Minnesota Twins': 'MIN', 'New York Mets': 'NYM',
    'New York Yankees': 'NYY', 'Athletics': 'ATH', 'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT', 'San Diego Padres': 'SD', 'San Francisco Giants': 'SF',
    'Seattle Mariners': 'SEA', 'St. Louis Cardinals': 'STL', 'Tampa Bay Rays': 'TB',
    'Texas Rangers': 'TEX', 'Toronto Blue Jays': 'TOR', 'Washington Nationals': 'WSH'
}

def format_to_local_time(utc_str, local_tz_str='US/Eastern'):
    """Helper to convert ISO UTC string to 12-hour local time."""
    if not utc_str:
        return "TBD"
    try:
        # Parse UTC string (e.g., 2025-06-15T18:10:00Z)
        utc_dt = datetime.fromisoformat(utc_str.replace('Z', '+00:00'))
        local_tz = pytz.timezone(local_tz_str)
        local_dt = utc_dt.astimezone(local_tz)
        return local_dt.strftime('%I:%M %p')
    except Exception:
        return "TBD"

def get_daily_starters(date_str, user_tz='US/Eastern'):
    # 1. Fetch all games for the specified date
    schedule = statsapi.schedule(date=date_str)
    
    if not schedule:
        return pd.DataFrame() # Return empty if no games
        
    games_list = []
    
    for game in schedule:
        # Convert UTC to Local Time using our helper
        raw_time = game.get("game_datetime")
        local_time = format_to_local_time(raw_time, user_tz)

        # Extract names and apply abbreviations
        home_full = game.get("home_name")
        away_full = game.get("away_name")

        game_data = {
            "Time": local_time,
            "Away": TEAM_ABBR.get(away_full, away_full),
            "Home": TEAM_ABBR.get(home_full, home_full),
            "Away Pitcher": game.get("away_probable_pitcher", "TBD"),
            "Home Pitcher": game.get("home_probable_pitcher", "TBD"),
            "Status": game.get("status"),
            "Venue": game.get("venue_name"),
            "game_id": game.get("game_id") # Keeping ID for background lookups
        }
        games_list.append(game_data)
    
    # 2. Convert to DataFrame
    df = pd.DataFrame(games_list)
    return df

# Test with June 15, 2025
df_starters = get_daily_starters('6/15/2025', 'US/Eastern')

# # Display result
# if not df_starters.empty:
#     print(df_starters[['Time', 'Away', 'Home', 'Away Pitcher', 'Home Pitcher']].head())
# else:
#     print("No games found for this date.")

Get today's lineup

In [ ]:
import statsapi
import pandas as pd

# 1. Get today's date automatically
today_str = datetime.now().strftime('%m/%d/%Y')

def get_lineups_to_df(date_str):
    # 1. Get all games for the day to find game_ids
    schedule = statsapi.schedule(date=date_str)
    
    lineup_records = []

    for game in schedule:
        game_id = game['game_id']
        team_names = {
            'home': game['home_name'],
            'away': game['away_name']
        }
        
        # 2. Pull boxscore data which contains the lineup
        try:
            box = statsapi.boxscore_data(game_id)
            
            for side in ['home', 'away']:
                # The 'battingOrder' list contains player IDs in order (1-9)
                order_ids = box[side].get('battingOrder', [])
                
                for slot, p_id in enumerate(order_ids, start=1):
                    player_info = box[side]['players'][f"ID{p_id}"]
                    
                    lineup_records.append({
                        "game_id": game_id,
                        "team": team_names[side],
                        "batting_order": slot,
                        "player_name": player_info['person']['fullName'],
                        "player_id": p_id,
                        "position": player_info['position']['abbreviation']
                    })
        except Exception as e:
            print(f"Lineup not yet available for {team_names['away']} @ {team_names['home']}")

    return pd.DataFrame(lineup_records)

# Test for a game day (using your June 15 example)
df_lineups = get_lineups_to_df('06/15/2025')
print(df_lineups.head(9)) # Show the lead-off through 9th hitter for the first game

In [ ]:
# Player's in the lineup that are due for a hit based on their luck score

# Assuming your Luck Score table is loaded as df_luck
df_today_luck = pd.merge(
    df_lineups, 
    luck_df[['player_id', 'luck_score', 'avg_exit_velo']], 
    on='player_id', 
    how='inner'
)

# Display the unluckiest players in today's lineup (due for a hit!)
due_for_hit = df_today_luck.sort_values('luck_score').head(5)

Get the standings by division

In [5]:
import statsapi
import pandas as pd

# Fetch raw standings data for both leagues
standings_raw = statsapi.standings_data(leagueId="103,104")

# Dictionary to store our DataFrames
division_dfs = {}

for div_id, div_data in standings_raw.items():
    div_name = div_data['div_name']
    teams_list = div_data['teams']
    
    # Convert the list of team dictionaries into a DataFrame
    df = pd.DataFrame(teams_list)
    
    # Clean up the DataFrame: keep only relevant columns
    cols_to_keep = ['name', 'w', 'l', 'pct', 'gb', 'wc_gb', 'streak']
    df = df[cols_to_keep]
    
    # Rename columns for better readability
    df.columns = ['Team', 'W', 'L', 'Win %', 'GB', 'WC GB', 'Streak']
    
    division_dfs[div_name] = df

# Example: Accessing a specific division
print("--- AL EAST ---")
print(division_dfs['American League East'])

KeyError: "['pct', 'streak'] not in index"

In [7]:
import statsapi

# Returns a dictionary structured by division
data = statsapi.standings_data(leagueId=103)

for division_id in data:
    division_name = data[division_id]['div_name']
    print(f"\n--- {division_name} ---")
    
    # Iterate through teams in the division
    for team in data[division_id]['teams']:
        name = team['name']
        w = team['w']
        l = team['l']
        gb = team['gb']
        print(f"{name}: {w}-{l} ({gb} GB)")


--- American League East ---
Toronto Blue Jays: 94-68 (- GB)
New York Yankees: 94-68 (- GB)
Boston Red Sox: 89-73 (5.0 GB)
Tampa Bay Rays: 77-85 (17.0 GB)
Baltimore Orioles: 75-87 (19.0 GB)

--- American League Central ---
Cleveland Guardians: 88-74 (- GB)
Detroit Tigers: 87-75 (1.0 GB)
Kansas City Royals: 82-80 (6.0 GB)
Minnesota Twins: 70-92 (18.0 GB)
Chicago White Sox: 60-102 (28.0 GB)

--- American League West ---
Seattle Mariners: 90-72 (- GB)
Houston Astros: 87-75 (3.0 GB)
Texas Rangers: 81-81 (9.0 GB)
Athletics: 76-86 (14.0 GB)
Los Angeles Angels: 72-90 (18.0 GB)


In [10]:
import statsapi

# Get raw data for one league
data = statsapi.standings_data(leagueId=103)

# Dig down to the first team in the first division found
first_division_key = list(data.keys())[0]
sample_team = data[first_division_key]['teams'][0]

# Print all available fields
print("Available Fields in 'teams' list:")
for field in sample_team.keys():
    print(f"- {field}")

Available Fields in 'teams' list:
- name
- div_rank
- w
- l
- gb
- wc_rank
- wc_gb
- wc_elim_num
- elim_num
- team_id
- league_rank
- sport_rank


This returns all the standings by division by League.
TODO need to add W%

In [ ]:
import statsapi
import pandas as pd

def get_division_standings_dfs():
    # leagueId 103 is AL, 104 is NL. Using both gets the whole league.
    standings_raw = statsapi.standings_data(leagueId="103,104")
    
    # This dictionary will hold { "Division Name": DataFrame }
    division_dict = {}

    for division_id in standings_raw:
        div_name = standings_raw[division_id]['div_name']
        teams_list = standings_raw[division_id]['teams']
        
        # Convert the list of team dicts directly to a DataFrame
        df = pd.DataFrame(teams_list)
        
        # Optional: Reorder or filter columns to keep it clean
        # TODO Calculate more columns if needed
        cols = ['name', 'w', 'l', 'gb', 'div_rank', 'wc_gb', 'wc_rank', 'league_rank']
        df = df[cols]
        
        # Set the Division Name as the key in the dictionary
        division_dict[div_name] = df
        
    return division_dict

# Execution
all_standings = get_division_standings_dfs()

# Example: Accessing just the NL West
print("--- National League West ---")
print(all_standings['National League West'])

--- National League West ---
                   name   w    l    gb div_rank wc_gb wc_rank league_rank
0   Los Angeles Dodgers  93   69     -        1     -       -           3
1      San Diego Padres  90   72   3.0        2  +7.0       2           5
2  San Francisco Giants  81   81  12.0        3   2.0       5           8
3  Arizona Diamondbacks  80   82  13.0        4   3.0       6           9
4      Colorado Rockies  43  119  50.0        5  40.0      12          15


In [13]:
def get_division_standings_dfs():
    """Fetches MLB standings, abbreviates team names, and formats for mobile."""
    
    TEAM_ABBR = {
        'Arizona Diamondbacks': 'ARI', 'Atlanta Braves': 'ATL', 'Baltimore Orioles': 'BAL',
        'Boston Red Sox': 'BOS', 'Chicago White Sox': 'CWS', 'Chicago Cubs': 'CHC',
        'Cincinnati Reds': 'CIN', 'Cleveland Guardians': 'CLE', 'Colorado Rockies': 'COL',
        'Detroit Tigers': 'DET', 'Houston Astros': 'HOU', 'Kansas City Royals': 'KC',
        'Los Angeles Angels': 'LAA', 'Los Angeles Dodgers': 'LAD', 'Miami Marlins': 'MIA',
        'Milwaukee Brewers': 'MIL', 'Minnesota Twins': 'MIN', 'New York Mets': 'NYM',
        'New York Yankees': 'NYY', 'Athletics': 'ATH', 'Philadelphia Phillies': 'PHI',
        'Pittsburgh Pirates': 'PIT', 'San Diego Padres': 'SD', 'San Francisco Giants': 'SF',
        'Seattle Mariners': 'SEA', 'St. Louis Cardinals': 'STL', 'Tampa Bay Rays': 'TB',
        'Texas Rangers': 'TEX', 'Toronto Blue Jays': 'TOR', 'Washington Nationals': 'WSH'
    }

    # leagueId 103 is AL, 104 is NL
    # standings_raw = statsapi.standings_data(leagueId="103,104") #This should be dynamic based on current season
    standings_raw = statsapi.standings_data(leagueId="103,104", season=2025)
    division_dict = {}

    for division_id in standings_raw:
        div_name = standings_raw[division_id]['div_name']
        teams_list = standings_raw[division_id]['teams']
        
        df = pd.DataFrame(teams_list)
        
        # Abbreviate names to save horizontal space
        df['name'] = df['name'].map(TEAM_ABBR).fillna(df['name'])
        
        # Calculate PCT and format as .XXX (MLB Style)
        df['W%'] = df['w'] / (df['w'] + df['l']).replace(0, 1)
        df['W%'] = df['W%'].apply(lambda x: '{:.3f}'.format(x).lstrip('0'))
        # cols = ['name', 'w', 'l', 'gb', 'div_rank', 'wc_gb', 'wc_rank', 'league_rank']
        
        # Map raw fields to clean display names
        cols_map = {
            'name': 'TM',
            'w': 'W',
            'l': 'L',
            'W%': 'PCT',
            'gb': 'GB',
            'wc_gb': 'WCGB',
            'league_rank': 'LG RNK'
        }
        
        df = df[list(cols_map.keys())].rename(columns=cols_map)
        division_dict[div_name] = df
        
    return division_dict

In [14]:
all_standings = get_division_standings_dfs()

In [4]:
standings_raw = statsapi.standings_data(leagueId="103", season=2025)

In [4]:
standings_raw = statsapi.standings_data(leagueId="103", season=2025, division="all")

In [15]:
import statsapi
import pandas as pd

def get_al_league_standings():
    """Fetches American League standings as a single DataFrame ordered by Rank."""
    
    TEAM_ABBR = {
        'Arizona Diamondbacks': 'ARI', 'Atlanta Braves': 'ATL', 'Baltimore Orioles': 'BAL',
        'Boston Red Sox': 'BOS', 'Chicago White Sox': 'CWS', 'Chicago Cubs': 'CHC',
        'Cincinnati Reds': 'CIN', 'Cleveland Guardians': 'CLE', 'Colorado Rockies': 'COL',
        'Detroit Tigers': 'DET', 'Houston Astros': 'HOU', 'Kansas City Royals': 'KC',
        'Los Angeles Angels': 'LAA', 'Los Angeles Dodgers': 'LAD', 'Miami Marlins': 'MIA',
        'Milwaukee Brewers': 'MIL', 'Minnesota Twins': 'MIN', 'New York Mets': 'NYM',
        'New York Yankees': 'NYY', 'Athletics': 'ATH', 'Philadelphia Phillies': 'PHI',
        'Pittsburgh Pirates': 'PIT', 'San Diego Padres': 'SD', 'San Francisco Giants': 'SF',
        'Seattle Mariners': 'SEA', 'St. Louis Cardinals': 'STL', 'Tampa Bay Rays': 'TB',
        'Texas Rangers': 'TEX', 'Toronto Blue Jays': 'TOR', 'Washington Nationals': 'WSH'
    }

    # Fetch data only for the American League (103)
    standings_raw = statsapi.standings_data(leagueId="103", season=2025)
    
    # Combine all teams from all AL divisions into one list
    all_al_teams = []
    for division_id in standings_raw:
        all_al_teams.extend(standings_raw[division_id]['teams'])
    
    # Create the single DataFrame
    df = pd.DataFrame(all_al_teams)
    
    # 1. Abbreviate names
    df['name'] = df['name'].map(TEAM_ABBR).fillna(df['name'])
    
    # 2. Calculate PCT (Standard MLB format)
    df['W%'] = df['w'] / (df['w'] + df['l']).replace(0, 1)
    df['W%_sort'] = df['W%'] # Keep a numeric version for sorting
    df['W%'] = df['W%'].apply(lambda x: '{:.3f}'.format(x).lstrip('0'))
    
    # 3. Map and Rename
    cols_map = {
        'name': 'TM',
        'w': 'W',
        'l': 'L',
        'W%': 'PCT',
        'gb': 'GB',
        'wc_gb': 'WCGB',
        'league_rank': 'LG RNK'
    }
    
    # Convert 'league_rank' to numeric to ensure correct sorting (1 before 10)
    df['league_rank'] = pd.to_numeric(df['league_rank'])
    
    # Sort by League Rank
    df = df.sort_values(by='league_rank', ascending=True)
    
    # Final filter and rename
    df = df[list(cols_map.keys())].rename(columns=cols_map)
    
    return df

# Usage in Streamlit
al_df = get_al_league_standings()
#st.table(al_df)

In [16]:
def get_league_standings(league_id):
    """Fetches American/National League standings as a single DataFrame ordered by Rank."""
    
    TEAM_ABBR = {
        'Arizona Diamondbacks': 'ARI', 'Atlanta Braves': 'ATL', 'Baltimore Orioles': 'BAL',
        'Boston Red Sox': 'BOS', 'Chicago White Sox': 'CWS', 'Chicago Cubs': 'CHC',
        'Cincinnati Reds': 'CIN', 'Cleveland Guardians': 'CLE', 'Colorado Rockies': 'COL',
        'Detroit Tigers': 'DET', 'Houston Astros': 'HOU', 'Kansas City Royals': 'KC',
        'Los Angeles Angels': 'LAA', 'Los Angeles Dodgers': 'LAD', 'Miami Marlins': 'MIA',
        'Milwaukee Brewers': 'MIL', 'Minnesota Twins': 'MIN', 'New York Mets': 'NYM',
        'New York Yankees': 'NYY', 'Athletics': 'ATH', 'Philadelphia Phillies': 'PHI',
        'Pittsburgh Pirates': 'PIT', 'San Diego Padres': 'SD', 'San Francisco Giants': 'SF',
        'Seattle Mariners': 'SEA', 'St. Louis Cardinals': 'STL', 'Tampa Bay Rays': 'TB',
        'Texas Rangers': 'TEX', 'Toronto Blue Jays': 'TOR', 'Washington Nationals': 'WSH'
    }

    # Fetch data only for the specified league
    standings_raw = statsapi.standings_data(leagueId=league_id, season=2025)
    
    # Combine all teams from all divisions into one list
    all_teams = []
    for division_id in standings_raw:
        all_teams.extend(standings_raw[division_id]['teams'])
    
    # Create the single DataFrame
    df = pd.DataFrame(all_teams)
    
    # 1. Abbreviate names
    df['name'] = df['name'].map(TEAM_ABBR).fillna(df['name'])
    
    # 2. Calculate PCT (Standard MLB format)
    df['W%'] = df['w'] / (df['w'] + df['l']).replace(0, 1)
    df['W%_sort'] = df['W%'] # Keep a numeric version for sorting
    df['W%'] = df['W%'].apply(lambda x: '{:.3f}'.format(x).lstrip('0'))
    
    # 3. Map and Rename
    cols_map = {
        'name': 'TM',
        'w': 'W',
        'l': 'L',
        'W%': 'PCT',
        'gb': 'GB',
        'wc_gb': 'WCGB',
        'league_rank': 'LG RNK'
    }
    
    # Convert 'league_rank' to numeric to ensure correct sorting (1 before 10)
    df['league_rank'] = pd.to_numeric(df['league_rank'])
    
    # Sort by League Rank
    df = df.sort_values(by='league_rank', ascending=True)
    
    # Final filter and rename
    df = df[list(cols_map.keys())].rename(columns=cols_map)
    
    return df

In [17]:
al_df = get_league_standings(103)
nl_df = get_league_standings(104)

In [ ]:
import statsapi
import pandas as pd

def get_game_results(date_str):
    TEAM_ABBR = {
    'Arizona Diamondbacks': 'ARI', 'Atlanta Braves': 'ATL', 'Baltimore Orioles': 'BAL',
    'Boston Red Sox': 'BOS', 'Chicago White Sox': 'CWS', 'Chicago Cubs': 'CHC',
    'Cincinnati Reds': 'CIN', 'Cleveland Guardians': 'CLE', 'Colorado Rockies': 'COL',
    'Detroit Tigers': 'DET', 'Houston Astros': 'HOU', 'Kansas City Royals': 'KC',
    'Los Angeles Angels': 'LAA', 'Los Angeles Dodgers': 'LAD', 'Miami Marlins': 'MIA',
    'Milwaukee Brewers': 'MIL', 'Minnesota Twins': 'MIN', 'New York Mets': 'NYM',
    'New York Yankees': 'NYY', 'Athletics': 'ATH', 'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT', 'San Diego Padres': 'SD', 'San Francisco Giants': 'SF',
    'Seattle Mariners': 'SEA', 'St. Louis Cardinals': 'STL', 'Tampa Bay Rays': 'TB',
    'Texas Rangers': 'TEX', 'Toronto Blue Jays': 'TOR', 'Washington Nationals': 'WSH'
    }
    
    # Fetch games for the specific date
    schedule = statsapi.schedule(date= date_str)
    
    if not schedule:
        return pd.DataFrame()

    results_list = []
    for game in schedule:
        # We only want games that are finished or in progress
        status = game.get("status", "")
        
        # Extract Scores
        home_score = int(game.get("home_score", 0))
        away_score = int(game.get("away_score", 0))

        # Extract names and apply abbreviations
        home_full = game.get("home_name")
        away_full = game.get("away_name")
        
        # Extract Winning/Losing Pitchers (only available if game is Final)
        winning_pitcher = game.get("winning_pitcher", "N/A")
        losing_pitcher  = game.get("losing_pitcher", "N/A")

        results_list.append({
            "Status": status,
            "Away": TEAM_ABBR.get(away_full, away_full),
            "R": away_score,
            "Home": TEAM_ABBR.get(home_full, home_full),
            "R ": home_score, # Space added to differentiate column name
            "Winner": winning_pitcher,
            "Loser": losing_pitcher,
            "Park": game.get("venue_name")
        })
    
    return pd.DataFrame(results_list)

def highlight_winner(row):
    # Default style (no background)
    style_away = ''
    style_home = ''
    
    # Highlight logic - only if the game is Final
    if "Final" in row['Status']:
        if row['R'] > row['R ']:
            style_away = 'background-color: #d4edda; font-weight: bold; border-radius: 4px;' # Light Green
        elif row['R '] > row['R']:
            style_home = 'background-color: #d4edda; font-weight: bold; border-radius: 4px;'
            
    return [None, style_away, None, style_home, None, None]

# Apply the function
results = get_game_results('6/15/2025')